# Polars

Es una biblioteca de procesamiento de datos rápida, eficiente y fácil de usar, diseñada para manejar grandes volúmenes de datos (en particular, en máquinas con memoria limitada) de manera mucho más eficiente que pandas, gracias a su uso de Rust y su capacidad para paralelizar operaciones.

Características clave:

* Lazy evaluation: Permite escribir código de manera declarativa y optimiza la ejecución de las operaciones a nivel de conjunto de datos.

* Paralelización automática: Polars puede aprovechar los núcleos del CPU para realizar operaciones en paralelo de manera transparente.

In [ ]:
import polars as pl
import pathlib

In [ ]:
# Se parametriza la ruta de los datos
PATH_DATA = pathlib.Path(r"../data")

## Lectura de ficheros

In [ ]:
# Lectura de parquet en DataFrame
df = pl.read_parquet(PATH_DATA.joinpath("prueba.parquet"))
df.head(2)

In [ ]:
# Lectura de parquet en LazyFrame
lf = pl.scan_parquet(PATH_DATA.joinpath("prueba.parquet"))
lf.head(2).collect()

In [ ]:
# Seleccionar columnas numericas por regex
# y mostrar su descriptivo
lf.select(pl.col(r"^(.*\d+)$")).describe()

In [ ]:
# Crear una nueva columna
lf = lf.with_columns((pl.col("COL_1") * 2).alias("COL_1_DOUBLED"))
lf = lf.with_row_index(name="INDEX").filter(
    (pl.col("INDEX") > 100) & (pl.col("INDEX") < 10000)
)
lf.head(2).collect(engine="streaming")

In [ ]:
# Renombrar columnas
lf = lf.rename({"INDEX": "INDICE"})
lf.head(2).collect()

In [ ]:
# Conseguir numero de columnas
lf.collect_schema().len()

##  Indexación y Slicing

In [ ]:
# Selección de columnas
lf.select(
    [
        (pl.col("COL_1") + pl.col("COL_2")).alias("COL_1_ADD_2"),
        (pl.col("COL_1") * pl.col("COL_2")).alias("COL_1_MULT_2"),
    ]
).head(2).collect()

In [ ]:
# Filtro
lf.filter((pl.col("INDICE") >= 0) & (pl.col("INDICE") < 1000)).head(2).collect()

In [ ]:
# Slicing
lf.select(pl.col("DATE_FIELD")).slice(1, 3).collect()

## Operaciones

In [ ]:
# Suma
lf.select((pl.col("COL_1") + pl.col("COL_2"))).head(2).collect()

In [ ]:
# Suma
lf.select(
    pl.sum_horizontal(pl.col("COL_1"), pl.col("COL_2")).alias("SUM_COL_1_COL_2")
).head(2).collect()

In [ ]:
# Calcular quantile
lf.select(pl.col("COL_1")).quantile(quantile=0.5).collect()

In [ ]:
# Calcular grupos de quantiles
lf.with_columns(
    pl.col("COL_1").qcut([0, 0.2, 0.5, 1], allow_duplicates=False).alias("QUANTILES")
).select([pl.col("COL_1"), pl.col("QUANTILES")]).head(2).collect()

## Group by

In [ ]:
# Group by
lf.group_by([pl.col("TYPE"), pl.col("COL_A")]).agg(
    [
        pl.col("COL_1").sum().alias("SUM_COL_1"),
        pl.col("COL_1").sum().alias("AVG_COL_1"),
    ]
).head(2).collect()

## Pivotar

In [ ]:
# Pivotar
lf_pivot = lf.group_by(pl.col("TYPE")).agg(
    [
        pl.col("COL_1").filter(pl.col("COL_A") == "REGION_3").mean().alias("A"),
        pl.col("COL_1").filter(pl.col("COL_A") == "REGION_1").mean().alias("B"),
    ]
)
lf_pivot.collect()

In [ ]:
# wide to long
lf_unpivot = lf_pivot.unpivot(
    index=["TYPE"], on=["A", "B"], variable_name="COL_A", value_name="VALUE"
)
lf_unpivot.collect()

## Concatenar

In [ ]:
# Concatenar
lf_concat = pl.concat([lf, lf.drop(pl.col("COL_1"))], how="diagonal")
print(lf_concat.select("COL_1").count().collect())
print(lf.select("COL_1").count().collect())
lf_concat.tail(2).collect()

## Ordenar

In [ ]:
# Ordenar
lf.sort(by=[pl.col("NRO_ID"), pl.col("DATE_FIELD")], descending=[True, True]).head(
    2
).collect()

## Joins

In [ ]:
# Crear LazyFrame auxiliar
dict_reg_ciu = {"REGION": [], "CIUDAD": []}
for i in range(20):
    dict_reg_ciu["REGION"].append(f"REGION_{i}")
    dict_reg_ciu["CIUDAD"].append(f"CIUDAD_{i}")
lf_reg_ciu = pl.LazyFrame(dict_reg_ciu)
lf_reg_ciu.head(2).collect()

In [ ]:
# Join para añadir variable CIUDAD
lf.join(
    lf_reg_ciu,
    how="inner",
    left_on=["COL_A"],
    right_on=["REGION"],
).head(2).collect()

## Escritura

In [ ]:
# Guardar parquet Particionado
partition = pl.PartitionMaxSize(
    PATH_DATA.joinpath("prueba2/part_{part}.parquet"), max_size=1_000
)  # 100_000_000 --> ~100 MB
lf.sink_parquet(
    partition,
    mkdir=True,
)

In [ ]:
# Guardar parquet Sin Particionar
lf.collect(engine="streaming").write_parquet(PATH_DATA.joinpath("prueba2.parquet"))